# Dependencies

In [1]:
import pandas as pd
import os
from itertools import permutations
import ast

# 物件設定

In [2]:
def input_preprocess(path):
    # load input 
    if not os.path.exists(path):
        raise IOError('No such file for input in the given path.')
    
    input_lines = []
    with open(path) as f:
        for line in f.readlines():
            input_lines.append(line.strip('\n'))

    # turn txt into a dict with correspond to columns in path table
    input_state = {
        'start_main': int(input_lines[0]),
        'end_main': int(input_lines[1])
    }
    try:
        for i in range(2, len(input_lines), 2):
            input_state['start_' + str(i//2 + 1)] = int(input_lines[i])
            input_state['path_' + str(i//2 + 1)] = ast.literal_eval(input_lines[i + 1])
    
    except ValueError:
        raise ValueError('Incorrect input format.') 
    
    return input_state

def compare_paths(path1, path2):
    # path1: from input, path2: from table
    if not path1:
        if not path2:
            return 1
        else:
            return 0
    else:
        if path2:

            edges1 = set(zip(path1[:-1], path1[1:]))
            edges2 = set(zip(path2[:-1], path2[1:]))
            intersection = edges1.intersection(edges2)
            # the similarity with comparison of path 1 
            # set operations are comparing edges
            # paths are in nodes, changed into edges by subtracting 1
            score = len(intersection)/(len(path1)-1)

            # penalty: the undesired edges in path2
            # proportion to the edges in path1 and path2
            diff = edges2.difference(intersection)
            score -= len(diff)/(len(path2)-1)/(len(path1)-1)
            return score
        else:
            return 0

In [3]:
class State_Compare(object):

    def __init__(self):
        self._original_table_list = None # became a list, for we need to read in chunks
        self._preprocessed = None
        self._ncars = None
        self._input_state = None
        self._start_end_filtered = None
        self._highest_similarity_with_original_data = None
        self._highest_similarity_and_reward = None
        
    def read_table(self, path, output=False):

        if not os.path.exists(path):
            raise IOError('No such file for loading table in the given path.')

        chunk_size = 50000
        df_list = []

        if '.csv' in path:

            with pd.read_csv(path, chunksize=chunk_size) as reader:
                for chunk in reader:
                    df_list.append(chunk)
            self._original_table_list = df_list

        elif '.json' in path:

            with pd.read_json(path, lines=True, orient='records', chunksize=chunk_size) as reader:
                for chunk in reader:
                    df_list.append(chunk)
            self._original_table_list = df_list
            self._preprocessed = True

        if output:
            return pd.concat(df_list, ignore_index=True)
        
    def set_ncars(self, ncars):

        self._ncars = ncars

    def set_input(self, input_state):

        self._input_state = input_state
        ncars = int((len(input_state)-2)/2 + 1)
        self.set_ncars(ncars)
    
    def preprocess(self, output=False):
        
        if self._preprocessed:
            # this may occur if read in table is of type json
            if output == True:
                return pd.concat(self._original_table_list, ignore_index=True)
            return        
        # this is to convert list as string into list
        # if data is stroed as csv
        if any(getattr(self, attr) is None for attr in [
            '_original_table_list', 
            '_ncars'
        ]):
            raise AttributeError('Attributes is not defined. Run read_table() first.')
        
        ncars_table = int((len(self._original_table_list[0].columns) - 1) / 3)

        if self._ncars != ncars_table:
            raise ValueError(''.join([
                'Ncars parameter is not aligned with columns in given table. Ncars: ', 
                str(self._ncars), 
                '. Ncars in table: ', 
                str(ncars_table), 
                '.'
            ]))
            
        for chunk in self._original_table_list:
            for i in range(2, self._ncars*2, 2):
                chunk[f'path_{i//2 + 1}'] = chunk[f'path_{i//2 + 1}'].apply(ast.literal_eval)

        self._preprocessed = True
        
        if output == True:
            return pd.concat(self._original_table_list, ignore_index=True)
            
        
    def filter_start_end(self, output=False):

        if any(getattr(self, attr) is None for attr in [
            '_ncars', 
            '_input_state', 
            '_preprocessed'
        ]):

            if self._preprocessed is None:
                raise AttributeError('Attribute preprocessed is not True. Run preprocess() first.')
            else:
                raise AttributeError('Attributes is not defined. Run set_input() and read_table() first.')
        
        subset_list = []
        for chunk in self._original_table_list:
            subset_list.append(chunk[
                (chunk['start_main'] == self._input_state['start_main']) & 
                (chunk['end_main'] == self._input_state['end_main'])
            ])

        # by this step, it should not be necessary to read in chunks
        self._start_end_filtered = pd.concat(subset_list, ignore_index=True)
        if output:
            return self._start_end_filtered
            

    def path_most_similar(self, output=False):

        # path has direction property
        if any(getattr(self, attr) is  None for attr in [
            '_ncars', 
            '_input_state', 
            '_start_end_filtered', 
            '_preprocessed'
        ]):
            if self._preprocessed is None:
                raise AttributeError('Attribute preprocessed is not True. Run preprocess() first.')
            elif self._start_end_filtered is None:
                raise AttributeError('Attribute subset is not defined. Run filter_start_end() first.')
            else:
                raise AttributeError('Attributes is not defined. Run set_input() and read_table() first.')
            
        # cars to compare = ncars - 1
        perm = permutations(range(0, self._ncars-1))
        perm_list = [list(p) for p in perm]
        len_perm = len(perm_list)

        path_cols  = []
        score_cols = []
        for i in range(1, self._ncars):
            path_cols.append('path_' + str(i+1))
            score_cols.append('path_similarity_' + str(i+1))
        subset_path = self._start_end_filtered[path_cols]
        input_paths = [self._input_state[key] for key in path_cols]

        path_similarity = pd.DataFrame(columns = score_cols.append('path_similarity_avg'))
        for i in range(len_perm):
            current_comparison = pd.DataFrame(columns=score_cols)
            for j in range(len(perm_list[i])):
                # for each permutation, compute a score of sum of each input path with correspond to table path
                current_comparison['path_similarity_' + str(j+2)] = subset_path.iloc[:, perm_list[i][j]].apply(lambda row: compare_paths(input_paths[j], row))
            current_comparison['path_similarity_avg'] = current_comparison.mean(axis=1)
            # extend the score df for each permutation
            if path_similarity.empty:
                path_similarity = current_comparison
            else:
                path_similarity = pd.concat([path_similarity, current_comparison])
        
        # find rows with max avg score in the score column
        highest_similarity = path_similarity[path_similarity['path_similarity_avg'] == path_similarity['path_similarity_avg'].max()]
        highest_similarity = highest_similarity.loc[highest_similarity.index.unique()]
        highest_similarity_idx = list(highest_similarity.index)
        # find the correspond row in the original table
        original_data_for_highest_similarity = self._start_end_filtered[self._start_end_filtered.index.isin(highest_similarity_idx)]
        highest_similarity_with_original_data = pd.concat([
            original_data_for_highest_similarity,
            highest_similarity            
        ], axis=1)
        self._highest_similarity_with_original_data = highest_similarity_with_original_data

        if output:
            return highest_similarity_with_original_data

            
    def highest_reward(self, output=False):

        if any(getattr(self, attr) is None for attr in [
            '_highest_similarity_with_original_data'
        ]):
            raise AttributeError('Attributes is not defined. Run path_most_similar() first.')
            

        # find rows with highest reward frim last step
        rows = self._highest_similarity_with_original_data
        highest_reward = rows[rows['reward'] == rows['reward'].max()]           

        self._highest_similarity_and_reward = highest_reward

        if output:
            return self._highest_similarity_and_reward

# 使用者介面

## 讀取前面生成的表格資料

In [4]:
# prepare table file
# execute time is mostly spent on reading in table
table = State_Compare()

input_state = input_preprocess('input.txt')
table.set_input(input_state)

table.read_table(path='simulation_data.csv')
table.preprocess(output=True)

,start_main,end_main,path_main,start_2,end_2,path_2,start_3,end_3,path_3,reward
0,13,2,"[13, 14, 1, 2]",13,2,[],13,2,[],0.965458
1,13,2,"[13, 14, 1, 2]",13,2,[],13,3,[],0.965458
2,13,2,"[13, 14, 1, 2]",13,2,[],13,4,[],0.965458
3,13,2,"[13, 14, 1, 2]",13,2,[],13,5,[],0.965458
4,13,2,"[13, 14, 1, 2]",13,2,[],13,9,[],0.965458
...,...,...,...,...,...,...,...,...,...,...
131067,12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",5,8,"[5, 6, 7, 8]",0.417962
131068,12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",9,8,"[9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",0.279793
131069,12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",10,8,"[10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",0.279793
131070,12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",12,8,"[12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",11,8,"[11, 12, 13, 14, 1, 2, 3, 4, 5, 6, 7, 8]",0.279793


## 讀取 input、拿 input 比對資料後找出 reward 最高的主機路徑（main_path）

In [5]:
# compare
table.filter_start_end()
table.path_most_similar()
table.highest_reward(output=True)

,start_main,end_main,path_main,start_2,end_2,path_2,start_3,end_3,path_3,reward,path_similarity_2,path_similarity_3,path_similarity_avg
24,5,8,"[5, 6, 7, 8]",13,2,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
88,5,8,"[5, 6, 7, 8]",13,3,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
152,5,8,"[5, 6, 7, 8]",13,4,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
216,5,8,"[5, 6, 7, 8]",13,5,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
280,5,8,"[5, 6, 7, 8]",13,9,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288,5,8,"[5, 6, 7, 8]",5,8,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
3352,5,8,"[5, 6, 7, 8]",9,8,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
3416,5,8,"[5, 6, 7, 8]",10,8,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
3480,5,8,"[5, 6, 7, 8]",11,8,[],13,2,"[13, 14, 1, 2]",0.991364,1.0,1,1.0
